# Natural Language Querying using Amazon Neptune and LlamaIndex

## Introduction

In this notebook we are going to demonstrate how you can leverage [LlamaIndex](https://www.llamaindex.ai/), and specifically the [Property Graph Index](https://docs.llamaindex.ai/en/stable/examples/property_graph/property_graph_basic/) feature to perform natural language querying with Amazon Neptune.  Let's start by taking a look at the two main components here: LlamaIndex and Natural Language Querying

LlamaIndex is a data structure and tooling designed to create and interact with large language model indexes. It facilitates the storage, searching, and querying of textual data using advanced vector database techniques in conjunction with large language models like GPT. This enables efficient and effective retrieval of relevant information from extensive text corpora.

Natural language querying is the ability to interact with computer systems using human language, rather than structured query languages or complex programming commands. It allows users to ask questions or provide instructions in their native language, and the system processes this input to understand the intent and provide relevant information or perform the requested action.  In the example here we will be using LlamaIndex to translate a natural language question into a structured graph query, specifically openCypher, which is then be executed on data in your Amazon Neptune database.

### Prerequisites

For this notebook we will be using Amazon Neptune Database as our data store so you must have a Neptune Database configured.  The methodology presented here will also work with Neptune Analytics and we will call out where the code differs.  This notebook will also require permissions to run Amazon Bedrock models, specifically `Claude v3 Sonnet` and `Titan Embedding v1`.

### Installing our dependencies

Run the next cell to install the core LlamaIndex packages as well as the specific package for Amazon Neptune and Amazon Bedrock

In [1]:
%pip install -q llama-index llama-index-graph-stores-neptune llama-index-llms-bedrock llama-index-embeddings-bedrock 

Note: you may need to restart the kernel to use updated packages.


### Loading your Data

The data we will use in this notebook is based on the data in *[Graph Databases in Action](https://www.manning.com/books/graph-databases-in-action?a_aid=bechberger)* by Manning Publications. The book uses the most common graph access patterns to build a fictitious application, DiningByFriends, that uses friends and ratings to provide personalized restaurant recommendations.

In the following notebook, we demonstrate how to use LlamaIndex and Amazon Neptune to build the queries for the DiningByFriends app using Natural Language questions instead of explicitly writing the queries.  


In [ ]:
%seed --model property_graph --language gremlin --dataset dining_by_friends --run

## Connecting to our Graph



In [4]:
from llama_index.graph_stores.neptune import NeptuneDatabasePropertyGraphStore
import graph_notebook as gn

# Retrieve the configuration of the notebook to get the current host
config = gn.configuration.get_config.get_config()

graph_store = NeptuneDatabasePropertyGraphStore(host=config.host)

In [5]:
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding


# Setup LlamaIndex to use Claude V3 Sonnet for the LLM
llm = Bedrock(model="anthropic.claude-3-sonnet-20240229-v1:0")

# Create the embedding model, this is required by the Property Graph
embed_model = BedrockEmbedding(model="amazon.titan-embed-text-v1")

Running the cell below will set the `llm` and `embed_model` globally.

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
from llama_index.core import PropertyGraphIndex
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    embed_model = embed_model,
    llm=llm
)

In [ ]:
index.property_graph_store.get_schema_str()

## Querying our Graph

In [11]:
from llama_index.core.indices.property_graph import TextToCypherRetriever

retriever = TextToCypherRetriever(index.property_graph_store, llm=llm)

nodes = retriever.retrieve("Who are Dave's Friends?")

for node in nodes:
    print(node.text)

Generated Cypher query:
MATCH (p:person {first_name: 'Dave'})-[:friends]->(friend:person)
RETURN friend.first_name, friend.last_name

Cypher Response:
[{'friend.first_name': 'Kelly', 'friend.last_name': 'Gorman'}, {'friend.first_name': 'Jim', 'friend.last_name': 'Miller'}, {'friend.first_name': 'Josh', 'friend.last_name': 'Perry'}, {'friend.first_name': 'Hank', 'friend.last_name': 'Erin'}]


In [12]:
nodes = retriever.retrieve("Are Dave and Denise connected?")

for node in nodes:
    print(node.text)

Generated Cypher query:
MATCH (p1:person {first_name: 'Dave'})-[:friends*..6]-(p2:person {first_name: 'Denise'})
RETURN p1, p2

Cypher Response:
[{'p1': {'~id': '10', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 1, 'last_name': 'Bech', 'first_name': 'Dave'}}, 'p2': {'~id': '45', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 8, 'last_name': 'Mande', 'first_name': 'Denise'}}}, {'p1': {'~id': '10', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 1, 'last_name': 'Bech', 'first_name': 'Dave'}}, 'p2': {'~id': '45', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 8, 'last_name': 'Mande', 'first_name': 'Denise'}}}, {'p1': {'~id': '10', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 1, 'last_name': 'Bech', 'first_name': 'Dave'}}, 'p2': {'~id': '45', '~entityType': 'node', '~labels': ['person'], '~properties': {'person_id': 8, 'last_name': 'Mande', 'first_n

In [ ]:
nodes = retriever.retrieve("Return the path of people connecting Dave and Denise?")
for node in nodes:
    print(node.text)

In [ ]:
nodes = retriever.retrieve("What are the newest reviews for a restaurant 'Southern Fire'?")

for node in nodes:
    print(node.text)

In [ ]:
nodes = retriever.retrieve("What restaurants near Dave with a diner or bar cuisine is the highest rated?")

for node in nodes:
    print(node.text)